In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.utils import to_categorical


In [2]:
# Закладка семян для воспроизводимости
np.random.seed(1234)  
PYTHONHASHSEED = 0

In [3]:
# Подготовка обучающей даты

# Чтение обучающей даты
train_df = pd.read_csv('./data/PM_train.txt', sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [4]:
# Подготовка тестировочной даты

# Чтение тестировочной даты
test_df = pd.read_csv('./data/PM_test.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [5]:
# Подготовка правдивой даты

# Чтение правдивой даты
truth_df = pd.read_csv('./data/PM_truth.txt', sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

In [6]:
# Создание столбца RUL для обучающих данных

# Создание DataFrame с максимальными значениями 'cycle' для каждого 'id'
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()

# Задание названий столбцов для DataFrame 'rul'
rul.columns = ['id', 'max']

# Слияние DataFrame 'train_df' с DataFrame 'rul' по столбцу 'id'
train_df = train_df.merge(rul, on=['id'], how='left')

# Создание столбца 'RUL', содержащего разницу между 'max' и 'cycle'
train_df['RUL'] = train_df['max'] - train_df['cycle']

# Удаление столбца 'max' из DataFrame 'train_df'
train_df.drop('max', axis=1, inplace=True)

In [7]:
train_df.head()
# Генерация столбцов меток для обучающих данных
w1 = 30  # Задание значения для порога w1
w0 = 15  # Задание значения для порога w0

# Создание столбца 'label1'. Если 'RUL' меньше или равно w1, то значение становится 1, иначе 0
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0)

# Создание столбца 'label2', который копирует значения из 'label1'
train_df['label2'] = train_df['label1']

# Обновление значений столбца 'label2'. Если 'RUL' меньше или равно w0, то значение становится 2
train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2

In [8]:
# MinMax нормализация обучающих данных
train_df['cycle_norm'] = train_df['cycle']  # Создание нового столбца 'cycle_norm', который копирует значения из столбца 'cycle'

# Определение столбцов для нормализации, исключая ненужные столбцы
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])

# Создание экземпляра MinMaxScaler для нормализации данных
min_max_scaler = preprocessing.MinMaxScaler()

# Нормализация выбранных столбцов с помощью метода fit_transform
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)

# Объединение нормализованных данных с исходными данными
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)

# Обновление DataFrame train_df с объединенными данными и переиндексацией столбцов
train_df = join_df.reindex(columns = train_df.columns)

In [9]:
# MinMax нормализация тренировочных данных

# Создание нового столбца 'cycle_norm', который копирует значения из столбца 'cycle' в test_df
test_df['cycle_norm'] = test_df['cycle']

# Нормализация тестовых данных test_df с помощью метода transform
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)

# Объединение нормализованных данных с оставшимися данными в test_df
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)

# Обновление DataFrame test_df с объединенными данными и переиндексацией столбцов
test_df = test_join_df.reindex(columns=test_df.columns)

# Сброс индексов источника данных для обучения и удаление старых индексов из test_df
test_df = test_df.reset_index(drop=True)

In [10]:
# Создание столбца RUL и столбцов меток для тестировочных данных

# Создание столбца 'max' для тестовых данных
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()

# Задание названий столбцов для DataFrame 'rul'
rul.columns = ['id', 'max']

# Изменение названия столбца в DataFrame 'truth_df'
truth_df.columns = ['more']

# Создание столбца 'id' в DataFrame 'truth_df' с помощью индексов и увеличение его значений на 1
truth_df['id'] = truth_df.index + 1

# Создание столбца 'max' в DataFrame 'truth_df', который является суммой 'max' из DataFrame 'rul' и столбца 'more' из DataFrame 'truth_df'
truth_df['max'] = rul['max'] + truth_df['more']

# Удаление столбца 'more' из DataFrame 'truth_df'
truth_df.drop('more', axis=1, inplace=True)

# Генерация столбца 'RUL' в test_df, который представляет разницу между 'max' и 'cycle'
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']

# Удаление столбца 'max' из DataFrame 'test_df'
test_df.drop('max', axis=1, inplace=True)

# Генерация столбцов меток 'label1' и 'label2' для тестовых данных, используя пороговые значения w0 и w1
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0)  # Значение 1, если 'RUL' меньше или равно w1, в противном случае 0
test_df['label2'] = test_df['label1']  # Копирование значений из 'label1' в 'label2'

# Обновление значений в столбце 'label2'. Если 'RUL' меньше или равно w0, то значение становится 2
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2

In [11]:
# Выбираем большой размер окна в 50 циклов
sequence_length = 50

In [12]:
# Функция для преобразования признаков в формат (образцы, временные шаги, признаки)
def gen_sequence(id_df, seq_length, seq_cols):
    """
    Функция gen_sequence принимает следующие аргументы:
    id_df: DataFrame, который содержит данные для конкретного идентификатора
    seq_length: длина последовательности
    seq_cols: список столбцов, используемых для создания последовательности
    """
    """ Рассматриваются только последовательности, которые соответствуют длине окна, без использования заполнения.
    Это означает, что при тестировании нам нужно исключить те, которые находятся ниже длины окна. 
    Альтернативой могло бы быть заполнение последовательностей, чтобы мы могли использовать более короткие """

    data_array = id_df[seq_cols].values  # Извлечение значений из DataFrame в массив
    num_elements = data_array.shape[0]  # Получение количества элементов в массиве

    # Цикл для создания последовательностей
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]
        # Генерация последовательности от start до stop из массива данных

In [13]:
# Определение столбцов признаков
sensor_cols = ['s' + str(i) for i in range(1,22)]  # Генерация списка столбцов 's1' до 's21'
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']  # Определение списка столбцов 'setting1', 'setting2', 'setting3' и 'cycle_norm'

sequence_cols.extend(sensor_cols)  # Добавление сгенерированных столбцов 's1' до 's21' к списку sequence_cols

In [14]:
# Генератор для последовательностей
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

In [15]:
# Генерация последовательностей и преобразование их в массив numpy
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)  # Объединение списка последовательностей в один массив numpy и преобразование в тип данных np.float32
seq_array.shape  # Вывод формы массива

(15631, 50, 25)

In [16]:
# Функция для генерации меток
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values  # Извлечение значений меток из DataFrame в массив
    num_elements = data_array.shape[0]  # Получение количества элементов в массиве
    return data_array[seq_length:num_elements, :]  # Возвращение значений меток, начиная с индекса seq_length и до конца массива

In [17]:
# Генерация меток
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label2']) 
             for id in train_df['id'].unique()]  # Генерация меток для каждого идентификатора в train_df

# Преобразование меток в формат one-hot encoding
label_array = to_categorical(np.concatenate(label_gen), num_classes=3)

# label_array = np.concatenate(label_gen).astype(np.float32)  # Объединение списка меток в один массив numpy и преобразование в тип данных np.float32
label_array.shape  # Вывод формы массива меток

(15631, 3)

In [18]:
# Построение нейронной сети
nb_features = seq_array.shape[2]  # Получение количества признаков из размерности seq_array
nb_out = label_array.shape[1]  # Получение количества выходных переменных из размерности label_array

model = Sequential()  # Инициализация последовательной модели

model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))  # Добавление слоя LSTM с 100 юнитами, возвращающего последовательности
model.add(Dropout(0.2))  # Добавление слоя Dropout для регуляризации

model.add(LSTM(
          units=50,
          return_sequences=False))  # Добавление слоя LSTM с 50 юнитами, не возвращающего последовательности
model.add(Dropout(0.2))  # Добавление слоя Dropout для регуляризации

model.add(Dense(units=nb_out, activation='softmax'))  # Добавление полносвязного слоя с активацией softmax
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # Компиляция модели с выбранной функцией потерь, оптимизатором и метриками


In [19]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 100)           50400     
                                                                 
 dropout (Dropout)           (None, 50, 100)           0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                30200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 3)                 153       
                                                                 
Total params: 80753 (315.44 KB)
Trainable params: 80753 (315.44 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [20]:
# Процесс обучения модели нейронной сети
model.fit(seq_array, label_array, epochs=10, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])
# seq_array: input-data / входные данные
# label_array: output-data / выходные данные; предсказания модели
# epochs: одна полная итерация по всему набору обучающих данных. Один проход через весь набор данных считается одной эпохой.
# batch_size: определяет количество образцов обучающих данных, которые будут переданы в сеть для обработки за один раз.
# validation_split=0.05: Доля данных, которая будет использоваться для проверки во время обучения. (Во время каждой эпохи процесс 
# обучения будет вычислять потери и метрики на валидационном наборе данных, чтобы оценить, как хорошо модель обобщает данные, которые она еще не видела.)
# verbose=1: В этом режиме модель будет выводить прогресс обучения для каждой эпохи, включая информацию о потерях и метриках.
# callbacks: Список обратных вызовов, которые могут выполняться во время обучения. В данном случае используется обратный вызов 
# EarlyStopping, который остановит обучение, если значение функции потерь на проверочном наборе данных не улучшается.

Epoch 1/10
75/75 [==============================] - 15s 144ms/step - loss: 0.3794 - accuracy: 0.8607 - val_loss: 0.2203 - val_accuracy: 0.8990
Epoch 2/10
75/75 [==============================] - 8s 113ms/step - loss: 0.1693 - accuracy: 0.9295 - val_loss: 0.1200 - val_accuracy: 0.9527
Epoch 3/10
75/75 [==============================] - 8s 113ms/step - loss: 0.1323 - accuracy: 0.9449 - val_loss: 0.1198 - val_accuracy: 0.9540
Epoch 4/10
75/75 [==============================] - 10s 131ms/step - loss: 0.1134 - accuracy: 0.9518 - val_loss: 0.1341 - val_accuracy: 0.9309


In [21]:

# Оценка производительности модели на обучающем наборе данных. Метод evaluate 
# вычисляет потери и метрики модели для переданных данных. В данном случае, seq_array 
# представляет собой входные данные, а label_array - соответствующие этим входным данным метки.

scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Accurracy: {}'.format(scores[1]))

79/79 [==============================] - 4s 46ms/step - loss: 0.1149 - accuracy: 0.9498
Accurracy: 0.9498432874679565


In [22]:
# Прогнозирование результатов и вычисление матрицы-путаницы на обучающих данных

# Матрица путаницы визуализирует точность классификатора, сравнивая фактические и прогнозируемые классы.
# Матрица двоичной путаницы состоит из квадратов:

#                       Predicted
#                   FALSE   |    TRUE
#           FALSE    (TN)   |    (FP)
#  Actual ------------------|--------------------------
#           TRUE     (FN)   |    (TP)

# TP: True Positive: прогнозируемые значения, правильно прогнозируемые как фактические положительные
# FP: Предсказанные значения неправильно предсказывают фактический положительный результат. т.е. отрицательные значения прогнозируются как положительные
# FN: False Negative: положительные значения прогнозируются как отрицательные
# TN: True Negative: прогнозируемые значения, правильно прогнозируемые как фактические негативные

# использование метода predict для получения вероятностей
y_pred_prob = model.predict(seq_array, batch_size=200)
y_pred = (y_pred_prob > 0.5).astype("float32")

y_pred_single = np.argmax(y_pred, axis=1)

# Сохранение истинных меток в y_true
y_true = label_array

y_true_single = np.argmax(y_true, axis=1)

# Вывод структуры матрицы неточностей
print('Матрица неточностей\n- По оси x находятся истинные метки.\n- По оси y находятся предсказанные метки')

# Вычисление матрицы неточностей с использованием функции confusion_matrix из библиотеки sklearn
cm = confusion_matrix(y_true_single, y_pred_single)

# Вывод матрицы неточностей
print(cm)


79/79 [==============================] - 4s 45ms/step
Матрица неточностей
- По оси x находятся истинные метки.
- По оси y находятся предсказанные метки
[[12278   253     0]
 [  267  1174    59]
 [    4   205  1391]]


In [23]:
combined_array = np.column_stack((y_true_single, y_pred_single))
combined_array

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [2, 2],
       [2, 2],
       [2, 2]], dtype=int64)

In [24]:
# Расчет метрик precision (точность) и recall (полнота)

# Precision (точность) - это метрика, которая оценивает долю правильно предсказанных 
# положительных классов от общего числа классифицированных как положительные. Формула для 
# вычисления precision: TP / (TP + FP), где TP - истинно положительные прогнозы, 
# а FP - ложно положительные прогнозы.

precision = precision_score(y_true_single, y_pred_single, average=None)

# Recall (полнота) - это метрика, которая оценивает долю правильно предсказанных положительных 
# классов от общего числа реальных положительных классов. Формула для вычисления recall: TP / (TP + FN), 
# где TP - истинно положительные прогнозы, а FN - ложно отрицательные прогнозы.

recall = recall_score(y_true_single, y_pred_single, average=None)

print( 'precision = ', precision, '\n', 'recall = ', recall)

precision =  [0.97840465 0.71936275 0.95931034] 
 recall =  [0.97981007 0.78266667 0.869375  ]


Проверим метрики модели на тестировочных данных. Будем производить предсказания на 50 последних временных рядах для каждого двигателя.

In [25]:
# Генерация последовательностей
sequence_length_test = 30
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length_test:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length_test]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

(100, 30, 25)

In [26]:
# Создаем маску, которая позволяет выбрать только те данные, для которых количество записей по
# идентификатору больше или равно значению переменной sequence_length.

y_mask = [len(test_df[test_df['id']==id]) >= sequence_length_test for id in test_df['id'].unique()]

# Генерация меток
label_array_test_last = test_df.groupby('id')['label2'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape

(100, 1)

In [27]:
label_array_test_last

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [2.],
       [2.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [2.],
       [0.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [2.],
       [0.],

In [28]:
# Оценка производительности модели на тестировочном наборе данных. Метод evaluate 
# вычисляет потери и метрики модели для переданных данных. В данном случае, seq_array_test_last 
# представляет собой входные данные, а label_array_test_last - соответствующие этим входным данным метки.
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Accurracy: {}'.format(scores_test[1]))

4/4 - 1s - loss: 3.5327 - accuracy: 0.8200 - 742ms/epoch - 186ms/step
Accurracy: 0.8199999928474426


In [29]:
# Прогнозирование результатов и вычисление матрицы-путаницы на тестировочных данных

# использование метода predict для получения вероятностей
y_pred_test_prob = model.predict(seq_array_test_last, batch_size=200)
# преобразование полученных вероятностей в бинарные предсказания с помощью сравнения с пороговым значением 0.5
y_pred_test = (y_pred_test_prob > 0.5).astype("float32")

y_pred_test_single = np.argmax(y_pred_test, axis=1)

# Сохранение истинных меток в y_true

y_true_test = label_array_test_last

# Вывод структуры матрицы неточностей
print('Матрица неточностей\n- По оси x находятся истинные метки.\n- По оси y находятся предсказанные метки')
cm = confusion_matrix(y_true_test, y_pred_test_single)
# Вывод матрицы неточностей
print(cm)

1/1 [==============================] - 1s 641ms/step
Матрица неточностей
- По оси x находятся истинные метки.
- По оси y находятся предсказанные метки
[[73  1  1]
 [ 9  3  3]
 [ 0  1  9]]


In [30]:
combined_array = np.column_stack((y_true_test, y_pred_test_single))
combined_array

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 2.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [2., 2.],
       [0., 0.],
       [0., 0.],
       [2., 2.],
       [2., 2.],
       [1., 2.],
       [1., 0.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [1., 0.],
       [2., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [1., 0.],
       [1., 0.],
       [0., 0.],
       [0., 0.],
       [2., 2.],
       [0., 0.],
       [0., 0.],
       [0., 0.